In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# 비디오 파일 열기
cap = cv2.VideoCapture(r"C:\Users\2580j\Downloads\MOV000248432_700X466.mp4")  # 여기서 'video.mp4'는 사용할 영상 파일명입니다.
data = []

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # 영상을 RGB로 변환하여 Mediapipe에 전달
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    

    if results.multi_hand_landmarks:
        print(f"Number of hands detected: {len(results.multi_hand_landmarks)}")
        all_d = []
        for res in results.multi_hand_landmarks:
            joint = np.zeros((21, 4))  # 21개의 관절에 대한 정보 저장 (x, y, z, visibility)

            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # Compute angles between joints
            v1 = joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19], :3]
            v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], :3]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            angle = np.arccos(np.einsum('nt,nt->n',
                                        v[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18], :],
                                        v[[1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19], :]))

            angle = np.degrees(angle)
            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, 1)  # Assuming idx is 1, change it accordingly

            d = np.concatenate([joint.flatten(), angle_label])
            all_d.append(d)
        data.append(np.array(all_d))

            # mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
    else:
        print("No hands detected in this frame")
    cv2.imshow('Hand Vector Data', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 저장된 데이터 확인
data = np.array(data)
print("Shape of data:", data.shape)

# 데이터를 파일로 저장 (예: CSV 형태로)
np.savetxt('hand_vector_data.csv', data, delimiter=',')  # 변경 가능한 파일명 및 형식입니다.

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [1]:
# 최종본
import cv2
import mediapipe as mp
import numpy as np
import time, os

actions = [
    'hello',
    'bread_house',
    'lunch',
    'NOISE'
]
seq_length = 30
secs_for_action = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
print(created_time)
os.makedirs(f'dataset/{created_time}', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        print(action)
        data = []
        ret, img = cap.read()
        img = cv2.flip(img, 1)

        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000)

        start_time = time.time()

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            if result.multi_hand_landmarks is not None:
                da = [] 
                if len(result.multi_hand_landmarks) == 2:
                    d = []
                    for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                        joint = np.zeros((21, 3))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z]

                        # Compute angles between joints
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                        v = v2 - v1 # [20, 3]
                        # Normalize v
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                        # Get angle using arcos of dot product
                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                        angle = np.degrees(angle) # Convert radian to degree
                        ############################################
                        angle = np.array([angle], dtype=np.float32)
                        d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                    da.append([np.concatenate(d)])
                    data.append(np.concatenate(da))        
                    
                    # da.append([np.concatenate(d),idx])
                elif len(result.multi_hand_landmarks)==1:
                    d= []
                    for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                        joint = np.zeros((21, 3))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z]

                        # Compute angles between joints
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                        v = v2 - v1 # [20, 3]
                        # Normalize v
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                        # Get angle using arcos of dot product
                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                        angle = np.degrees(angle) # Convert radian to degree

                        angle = np.array([angle], dtype=np.float32)
                        d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                        
                    d.append(np.zeros_like(d[0]))
                    da.append([np.concatenate(d)])  
                    data.append(np.concatenate(da))        

                
                # mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[-1])
        full_seq_data = []
        # print(data.shape)
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape, data.shape)
        # if fs == 0:
        #     fs = full_seq_data
        #     da = data
        if len(full_seq_data.shape) ==3 :
            np.save(os.path.join(f'dataset/{created_time}', f'raw_{action}'), data)
            np.save(os.path.join(f'dataset/{created_time}', f'seq_{action}_{full_seq_data.shape}'), full_seq_data)
            print(action,'saved')
    cv2.destroyAllWindows()
    break
if os.path.exists(f'dataset/{created_time}') and not os.listdir(f'dataset/{created_time}'):
    os.rmdir(f'dataset/{created_time}')  
    print(f"{f'dataset/{created_time}'} 빈폴더 삭제")

1705043857
dataset/1705043857 빈폴더 삭제


In [2]:
#  비디오 학습
import cv2
import mediapipe as mp
import numpy as np
import time, os

actions = [
    'hello',
    'bread_house',
    'lunch',
    'NOISE'
]
seq_length = 30
secs_for_action = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(r"C:\Users\2580j\Downloads\MOV000248432_700X466.mp4")

created_time = int(time.time())
print(created_time)
os.makedirs(f'dataset/{created_time}', exist_ok=True)

while cap.isOpened():
    for idx, action in enumerate(actions):
        print(action)
        data = []
        ret, img = cap.read()

        start_time = time.time()

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            # img = cv2.flip(img, 1) ### 반전 사용 X 2
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            if result.multi_hand_landmarks is not None:
                da = [] 
                if len(result.multi_hand_landmarks) == 2:
                    d = []
                    for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                        joint = np.zeros((21, 3))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z]

                        # Compute angles between joints
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                        v = v2 - v1 # [20, 3]
                        # Normalize v
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                        # Get angle using arcos of dot product
                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                        angle = np.degrees(angle) # Convert radian to degree
                        ############################################
                        angle = np.array([angle], dtype=np.float32)
                        d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                    da.append([np.concatenate(d)])
                    data.append(np.concatenate(da))        
                    
                    # da.append([np.concatenate(d),idx])
                elif len(result.multi_hand_landmarks)==1:
                    d= []
                    for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                        joint = np.zeros((21, 3))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z]

                        # Compute angles between joints
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                        v = v2 - v1 # [20, 3]
                        # Normalize v
                        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                        # Get angle using arcos of dot product
                        angle = np.arccos(np.einsum('nt,nt->n',
                            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                        angle = np.degrees(angle) # Convert radian to degree

                        angle = np.array([angle], dtype=np.float32)
                        d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                        
                    d.append(np.zeros_like(d[0]))
                    da.append([np.concatenate(d)])  
                    data.append(np.concatenate(da))        

                
                # mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        data = np.array(data)
        data = data.reshape(data.shape[0],data.shape[-1])
        full_seq_data = []
        # print(data.shape)
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape, data.shape)
        # if fs == 0:
        #     fs = full_seq_data
        #     da = data
        if len(full_seq_data.shape) ==3 :
            np.save(os.path.join(f'dataset/{created_time}', f'raw_{action}'), data)
            np.save(os.path.join(f'dataset/{created_time}', f'seq_{action}_{full_seq_data.shape}'), full_seq_data)
            print(action,'saved')
    cv2.destroyAllWindows()
    break
if os.path.exists(f'dataset/{created_time}') and not os.listdir(f'dataset/{created_time}'):
    os.rmdir(f'dataset/{created_time}')  
    print(f"{f'dataset/{created_time}'} 빈폴더 삭제")

1705315708
hello
hello (173, 30, 156) (203, 156)
hello saved
bread_house
bread_house (35, 30, 156) (65, 156)
bread_house saved
lunch
lunch (0,) (4, 156)
NOISE
NOISE (0,) (5, 156)


In [43]:
#학습 수정본
import cv2
import mediapipe as mp
import numpy as np
import time, os

action =     'hello'
seq_length = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

created_time = int(time.time())
print(created_time)
os.makedirs(f'dataset/{created_time}', exist_ok=True)

while cap.isOpened():
    data = []
    while True:
        ret, img = cap.read()

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks is not None:
            da = [] 
            if len(result.multi_hand_landmarks) == 2 or len(result.multi_hand_landmarks) == 1:
                d = []
                for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                    joint = np.zeros((21, 3))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                    angle = np.degrees(angle) # Convert radian to degree
                    angle = np.array([angle], dtype=np.float32)
                    d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                    if len(result.multi_hand_landmarks)==1:
                        d.append(np.zeros_like(d[0]))
                da.append([np.concatenate(d)])
                data.append(np.concatenate(da))        


        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break

    data = np.array(data)
    data = data.reshape(data.shape[0],data.shape[-1])
    full_seq_data = []
    # print(data.shape)
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])

    full_seq_data = np.array(full_seq_data)
    print(action, full_seq_data.shape, data.shape)

    if len(full_seq_data.shape) ==3 :
        np.save(os.path.join(f'dataset/{created_time}', f'raw_{action}'), data)
        np.save(os.path.join(f'dataset/{created_time}', f'seq_{action}_{full_seq_data.shape}'), full_seq_data)
        print(action,'saved')
    cv2.destroyAllWindows()
    break
if os.path.exists(f'dataset/{created_time}') and not os.listdir(f'dataset/{created_time}'):
    os.rmdir(f'dataset/{created_time}')  
    print(f"{f'dataset/{created_time}'} 빈폴더 삭제")

0
1
2
3
4


In [49]:
import cv2

# 원격 비디오 URL
video_url = "http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20221014/1040361/MOV000360760_700X466.mp4"

# VideoCapture 객체 생성
cap = cv2.VideoCapture(video_url)
# 동영상의 프레임 수와 크기 가져오기
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(fps, width,height)
# 비디오가 열렸는지 확인
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # 비디오 프레임 읽기
    while True:
        ret, frame = cap.read()
        
        # 비디오가 끝나면 종료
        if not ret:
            break

        # 여기에서 프레임에 대한 추가 처리 수행 가능

        # 프레임 화면에 표시
        cv2.imshow('Video', frame)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # VideoCapture 객체 해제
    cap.release()

    # 창 닫기
    cv2.destroyAllWindows()


59 700 466


In [54]:
#  비디오 학습
import cv2
import mediapipe as mp
import numpy as np
import time, os
import random


##############################

def rotate_image(image, angle):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.5)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

#변수
VIDEO_PATH = "http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20221014/1040361/MOV000360760_700X466.mp4"

ACTION = 'hello'
seq_length = 30
created_time = int(time.time())
print(created_time)
os.makedirs(f'dataset/{ACTION}', exist_ok=True)

# 동영상 파일 열기
cap = cv2.VideoCapture(VIDEO_PATH)

# 동영상이 제대로 열렸는지 확인
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# 동영상의 프레임 수와 크기 가져오기
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

rotate_li = [0, 10, -10, 15, -15]
speed_li = [1, 2, 3, 4, 5]
random.shuffle(speed_li) 

###############################

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

data = []
for repeat in range(len(rotate_li)):
    speed = speed_li[repeat]
    rotate = rotate_li[repeat]
    
    print(repeat,'번째 실행입니다.', f'speed : {speed}, rotated : {rotate}')
    
    while True:
        ret, img = cap.read()
        if not ret: # 영상끝나면 종료
            break
        img = rotate_image(img, rotate)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks is not None:
            da = [] 
            if len(result.multi_hand_landmarks) == 2 or len(result.multi_hand_landmarks) == 1:
                d = []
                for res in result.multi_hand_landmarks:  # res 잡힌 만큼 (max 손 개수 이하)
                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
                    joint = np.zeros((21, 3))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]\

                    angle = np.degrees(angle) # Convert radian to degree
                    angle = np.array([angle], dtype=np.float32)
                    d.append(np.concatenate([joint.flatten(),angle.flatten()]))
                    if len(result.multi_hand_landmarks)==1:
                        d.append(np.zeros_like(d[0]))
                da.append([np.concatenate(d)])
                data.append(np.concatenate(da))        


        cv2.imshow('img', img)
        if cv2.waitKey(int(30 / speed)) & 0xFF == ord('q'):
            break
        # 동영상 속도에 따라 프레임 위치 설정
        frame_index = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) + speed
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    
    # 동영상 다시재생
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    
    
# 배열화
data = np.array(data)
data = data.reshape(data.shape[0],data.shape[-1])

# 시쿼스 분리
full_seq_data = []
for seq in range(len(data) - seq_length):
    full_seq_data.append(data[seq:seq + seq_length])

full_seq_data = np.array(full_seq_data)
print(ACTION, full_seq_data.shape, data.shape) # 데이터 모양 확인

# 파일 저장
if len(full_seq_data.shape) ==3 :
    np.save(os.path.join(f'dataset/{ACTION}', f'raw_{created_time}'), data)
    np.save(os.path.join(f'dataset/{ACTION}', f'seq_{created_time}_{full_seq_data.shape[0]}'), full_seq_data)
    print(ACTION,'saved')
cv2.destroyAllWindows()

# 비정상 폴더 삭제 (빈폴더)
if os.path.exists(f'dataset/{ACTION}') and not os.listdir(f'dataset/{ACTION}'):
    os.rmdir(f'dataset/{ACTION}')  
    print(f"{f'dataset/{ACTION}'} 빈폴더 삭제")

1705326983
0 번째 실행입니다. speed : 5, rotated : 0
1 번째 실행입니다. speed : 4, rotated : 10
2 번째 실행입니다. speed : 1, rotated : -10
3 번째 실행입니다. speed : 2, rotated : 15
4 번째 실행입니다. speed : 3, rotated : -15
hello (146, 30, 156) (176, 156)
hello saved


In [ ]:
import json

# JSON 파일 읽기
with open('data.json', 'r', encoding='utf-8') as json_file:
    loaded_data = json.load(json_file)

# 읽어온 데이터 출력
print('JSON 파일에서 읽어온 딕셔너리:')
print(loaded_data)

# 특정 키에 접근하여 값 얻기
name = loaded_data['이름']
age = loaded_data['나이']
print(f'이름: {name


In [15]:
import json
from datetime import datetime

def page_todo(file_path, key):
    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    except FileNotFoundError:

        data = {}

    if key not in data: # key 없을때 

        data[key] = dict()


        with open(file_path, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
            print(f"{file_path}에 키 '{key}'가 없어서 추가하였습니다.")
            return 1
    else: # 키 있으면 해당키의 최근 페이지 반환
        return max(data[key]) +1 # 있으면 다음페이지 반환
        

def write_txt_log(T_path, text):
    current_time = datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

    with open(T_path, 'w') as file:
        file.write(current_time, ':::',text,'\n')

def write_json_log(J_PATH, data):
    with open(J_PATH, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

In [16]:
from datetime import datetime

def get_current_time():
    # 현재 시간을 얻기
    current_time = datetime.now()

    # 시간을 문자열로 포맷팅
    formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

    return formatted_time

# 현재 시간을 반환받기
current_time = get_current_time()

# 출력
print(f'현재 시간: {current_time}')


현재 시간: 2024-01-17 17:09:19


In [7]:
import requests
import xmltodict
import json


todo_page = page_todo('api_log.json', 'Daily')

url = 'http://api.kcisa.kr/openapi/service/rest/meta13/getCTE01701'
params = {
    'serviceKey': 'ecc7282e-731e-4aa0-91b1-017535926c8f',
    'numOfRows': 10,
    'pageNo': todo_page,
}

# GET 요청 보내기
response = requests.get(url, params=params)

# 서버 응답 코드 확인
if response.status_code == 200:
    print('요청 성공')
    write_txt_log('LOG.TXT', f'Page {page_todo} api 요청 성공')

    # 서버에서 반환한 Content-Type 확인
    content_type = response.headers.get('Content-Type')

    # Content-Type이 존재하고 'charset'이라는 단어가 들어있는 경우
    if content_type and 'charset' in content_type:
        encoding = content_type.split('charset=')[-1].strip()
    else:
        # Content-Type이 없거나 'charset'이라는 단어가 없는 경우 기본값으로 utf-8 설정
        encoding = 'utf-8'

    # XML을 파이썬 사전으로 변환
    xml_dict = xmltodict.parse(response.text, encoding=encoding)

    # 파이썬 사전을 JSON으로 변환
    json_response = json.dumps(xml_dict, ensure_ascii=False, indent=2)
    # print('응답 내용:', json_response)
            
    item_li = json_response['response']['body']['items']['item']
    data_to_log = {}
    for item in item_li:
        data_to_log[todo_page][item['title']] = [item['title'],item['subDescription']]
else:
    print(f'요청이 실패했습니다. 응답 코드: {response.status_code}')
    write_txt_log('LOG.TXT', f'Page {page_todo} api 요청 실패\n 응답 코드: {response.status_code}')

요청 성공
응답 내용: {
  "response": {
    "header": {
      "resultCode": "0000",
      "resultMsg": "OK"
    },
    "body": {
      "items": {
        "item": [
          {
            "title": "끈적끈적",
            "alternativeTitle": null,
            "creator": "누리집",
            "regDate": "2023-05-24 11:04:42",
            "collectionDb": "일상생활수어",
            "subjectCategory": null,
            "subjectKeyword": null,
            "extent": null,
            "description": null,
            "spatialCoverage": null,
            "temporal": null,
            "person": null,
            "language": null,
            "sourceTitle": null,
            "referenceIdentifier": "http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201211/785169/MOV000357015_215X161.jpg",
            "rights": null,
            "copyrightOthers": null,
            "url": "http://sldict.korean.go.kr/front/sign/signContentsView.do?top_category=CTE&origin_no=4704",
            "contributor": null,
        

In [2]:
import json
a = json.loads(json_response)
a

{'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'},
  'body': {'items': {'item': [{'title': '끈적끈적',
      'alternativeTitle': None,
      'creator': '누리집',
      'regDate': '2023-05-24 11:04:42',
      'collectionDb': '일상생활수어',
      'subjectCategory': None,
      'subjectKeyword': None,
      'extent': None,
      'description': None,
      'spatialCoverage': None,
      'temporal': None,
      'person': None,
      'language': None,
      'sourceTitle': None,
      'referenceIdentifier': 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201211/785169/MOV000357015_215X161.jpg',
      'rights': None,
      'copyrightOthers': None,
      'url': 'http://sldict.korean.go.kr/front/sign/signContentsView.do?top_category=CTE&origin_no=4704',
      'contributor': None,
      'subDescription': 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201211/785169/MOV000357015_700X466.mp4'},
     {'title': '나그네',
      'alternativeTitle': None,
      'crea

In [3]:
a['response']['body']['items']['item'][0]['subDescription']

'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201211/785169/MOV000357015_700X466.mp4'

In [4]:
response = a

In [5]:
from data_generator import *

for i in response['response']['body']['items']['item']:
    make_data(i['title'],i['subDescription'] )

created at : 1705327617
0 번째 실행입니다. speed : 3, rotated : 0
1 번째 실행입니다. speed : 4, rotated : 10
2 번째 실행입니다. speed : 1, rotated : -10
3 번째 실행입니다. speed : 5, rotated : 15
4 번째 실행입니다. speed : 2, rotated : -15
끈적끈적 saved (166, 30, 156) (196, 156)
created at : 1705327698
0 번째 실행입니다. speed : 5, rotated : 0
1 번째 실행입니다. speed : 2, rotated : 10
2 번째 실행입니다. speed : 3, rotated : -10
3 번째 실행입니다. speed : 4, rotated : 15
4 번째 실행입니다. speed : 1, rotated : -15
나그네 saved (143, 30, 156) (173, 156)
created at : 1705327780
0 번째 실행입니다. speed : 2, rotated : 0
1 번째 실행입니다. speed : 3, rotated : 10
2 번째 실행입니다. speed : 1, rotated : -10
3 번째 실행입니다. speed : 5, rotated : 15
4 번째 실행입니다. speed : 4, rotated : -15
낙지 saved (73, 30, 156) (103, 156)
created at : 1705327816
0 번째 실행입니다. speed : 3, rotated : 0
1 번째 실행입니다. speed : 2, rotated : 10
2 번째 실행입니다. speed : 5, rotated : -10
3 번째 실행입니다. speed : 4, rotated : 15
4 번째 실행입니다. speed : 1, rotated : -15
감추다,숨다,은닉 saved (76, 30, 156) (106, 156)
created at : 1705327875
0 번째 실행입